In [1]:
import dendropy
from dendropy import Tree, TaxonNamespace
import pyfasta
from pyfasta import Fasta
import re
import os
from os import path
import shutil

In [20]:
#remove any alignments that don't start with ATG

alignment_dir = '/Users/clairedubin/spur/small_ortho_aligned_031420/'


count = 0
total = 0

bad = []

for file in os.listdir(alignment_dir):    
    
    if file[-2:] == 'fa':

        total += 1

        f=Fasta(alignment_dir+file)

        sample = list(f.keys())[0]

        if len(f[sample][:]) % 3 != 0:

            
            for key in f.keys():
                
                for codon in range(0, len(sample), 3):
                    
                    if '-' not in f[key][codon:codon+3]:
                        
                        if f[key][codon:codon+3] != 'ATG':
                            

                            
                            bad += [file]

                            
                        break
                                                  
        
print(len(set(bad)), total)

for file in bad:
    if os.path.exists(alignment_dir+file):
        os.remove(alignment_dir+file)
    

155 4782


In [39]:
#use files with at least 10 strains

good_files = []

count = 0
total = 0
for file in os.listdir(alignment_dir):
    
    if file[-2:] == 'fa':
        total +=1
        f = Fasta(alignment_dir+file)

        if len(f.keys()) >= 10:

            count+=1 
            
            good_files += [file]

print(count, total)

3498 4627


In [40]:
#make all files have sequence length % 3 = 0 (PAML won't run otherwise)

for file in good_files:
        
    if file[-2:] == 'fa':

        f=Fasta(alignment_dir+file)

        sample = list(f.keys())[0]

        i = len(f[sample][:]) % 3

        if i != 0:
            
            to_write = ''

            for key in f.keys():
                
                to_write += '>' + key + '\n' + f[key][:-i] + '\n'

            p = open('/Users/clairedubin/spur/temp/'+file, 'w')
            p.write(to_write)
            p.close()
            
for file in os.listdir('/Users/clairedubin/spur/temp/'):

    shutil.copy('/Users/clairedubin/spur/temp/'+file, alignment_dir + file)

In [2]:
def make_tree(file, master_path, output_path):
    
    gene = file.split('.')[0]
    
    f = Fasta(file)
    
    keys_to_keep = [f.replace('_', ' ') for f in list(f.keys())]
        
    master = Tree.get(file=open(master_path, 'r'),
                schema="newick",
                tree_offset=0)
        
    master.retain_taxa_with_labels(keys_to_keep)
    
    master_string = master.as_string(schema='newick')
    
    master_string = re.sub(r'\:[\d.]+', '', master_string)
        
    species_count = len(master_string.split(','))
                     
    master_string = str(species_count) + ' 1\n' + master_string
    
    newfile = open(output_path, 'w')
    
    newfile.write(master_string)
    
    newfile.close()
    
    return master_string


In [34]:
def make_tree_promoters(file, master_path, output_path):
    
    gene = file.split('.')[0]
    
    f = Fasta(file)
    
    keys_to_keep = [f.split('_', 1)[1].capitalize().replace('_', ' ') for f in list(f.keys())]
        
    master = Tree.get(file=open(master_path, 'r'),
                schema="newick",
                tree_offset=0)
        
    master.retain_taxa_with_labels(keys_to_keep)
    
    master_string = master.as_string(schema='newick')
    
    master_string = re.sub(r'\:[\d.]+', '', master_string)
        
    species_count = len(master_string.split(','))
                     
    master_string = str(species_count) + ' 1\n' + master_string
    
    newfile = open(output_path, 'w')
    
    newfile.write(master_string)
    
    newfile.close()
    
    return master_string

In [35]:
go5778 = ['YPL112C',
 'YHR150W',
 'YPR165W',
 'YGR028W',
 'YNL214W',
 'YPR128C',
 'YML075C',
 'YGR004W',
 'YGR239C',
 'YLR324W',
 'YGL153W',
 'YHR160C',
 'YMR163C',
 'YDR265W',
 'YLR450W',
 'YDR244W',
 'YMR026C',
 'YKL188C',
 'YDR329C',
 'YLR191W',
 'YDR479C',
 'YOL147C',
 'YOR193W',
 'YBR168W',
 'YPL147W',
 'YBR222C',
 'YNL329C',
 'YDL065C',
 'YOL044W',
 'YAL055W',
'YJL210W',
'YKL197C',
'YMR018W']

In [36]:
len(go5778)

33

In [37]:
#make one tree per gene (promoters)

master_path = "/Users/clairedubin/Downloads/pruned_sacc_tree.txt"

alignment_dir = '/Users/clairedubin/spur/peroxisomes/PAML_for_reviewers_031820/sacc_larger_phylogeny/promoters_aligned/'

for file in os.listdir(alignment_dir):
    
    output_path = '/Users/clairedubin/spur/peroxisomes/PAML_for_reviewers_031820/sacc_larger_phylogeny/promoter_trees/' + file.split('.')[0] + '.tree'
    
    make_tree_promoters(alignment_dir+file, master_path, output_path)

In [45]:
#make one tree per gene

master_path = "/Users/clairedubin/Downloads/pruned_sacc_tree.txt"

for file in good_files:
    
    output_path = '/Users/clairedubin/spur/genome_trees_031620/' + file.split('.')[0] + '.tree'
    
    make_tree(alignment_dir+file, master_path, output_path)

In [48]:
len(set(bad))

2543

In [57]:
all_keys = list(Fasta('/Users/clairedubin/spur/small_ortho_aligned_031420/YPL112C.muscle_afa').keys())
count = 0

for gene in go5778:
    if os.path.exists('/Users/clairedubin/spur/small_ortho_aligned_031420/' + gene + '.muscle_afa'):
        f = Fasta('/Users/clairedubin/spur/small_ortho_aligned_031420/' + gene + '.muscle_afa')
        
        
                
        if len(f.keys()) >= 10:
            count+=1

            print(gene, len(f.keys()))

            for key in all_keys:

                if key not in list(f.keys()):

                    print(key)
    else:
        print(gene, 'no')

YPL112C 13
YHR150W 13
YPR165W 13
YGR028W 13
YPR128C 12
Nakaseomyces_castellii
YML075C 13
YGR004W 13
YGR239C 11
Nakaseomyces_castellii
Nakaseomyces_bacillisporus
YLR324W no
YGL153W 10
Saccharomyces_uvarum
Saccharomyces_arboricola
Saccharomyces_mikatae
YMR163C 10
Saccharomyces_uvarum
Saccharomyces_arboricola
Saccharomyces_mikatae
YDR265W 12
Nakaseomyces_castellii
YLR450W no
YDR244W 13
YMR026C 13
YKL188C 13
YLR191W 13
YDR479C 13
YBR168W 13
YPL147W 13
YBR222C 13
YNL329C 10
Saccharomyces_uvarum
Saccharomyces_arboricola
Saccharomyces_paradoxus
YDL065C 13
YOL044W 10
Nakaseomyces_castellii
Nakaseomyces_bacillisporus
Saccharomyces_paradoxus
